In [1]:
import pandas as pd
import operator
import numpy as np
from nltk import ngrams
import seaborn as sns
import nltk
from nltk.stem.snowball import SnowballStemmer
%matplotlib inline

In [38]:
df = pd.read_csv('../data/lable_stammed.csv')

In [3]:

manual = pd.read_csv('../data/manual.csv')

In [41]:
len(df[df.label == True])

5861

In [42]:
len(df)

46023

In [43]:
df = df.dropna()

In [44]:
def delete_short_words(x):
    new_string = ' '.join([w for w in x.split() if len(w)>2])
    return new_string

In [45]:
df['text'] = df['text'].apply(delete_short_words)

In [46]:
df.head(1)

,id,video_id,time,author,text,label
0,UgygtiZyJIQoKYMr5-14AaABAg,_k-gDXrfu-s,4 days ago,Taki Mizu,нужно было града всю толпу поливать,False


In [47]:
df = df[~df.id.isin(manual.id)]

In [48]:
len(df)

45190

In [49]:
stemmer = SnowballStemmer("russian", ignore_stopwords=True) 

In [50]:
def stem(word):
    stemmer = SnowballStemmer("russian", ignore_stopwords=True) 
    res = stemmer.stem(word)
    if len(res) <= 2:
        return word
    return res

In [51]:
bad_words = open('../data/bad_words_seed.txt').read().split('\n')[:-1]

In [52]:
report_dictionary = []
stemmed_dictionary = []

In [53]:
bad_words

['бля', 'сука', 'пизда', 'ебать', 'хуй']

In [54]:
for word in bad_words:
    report_dictionary.append(word)
    stemmed_dictionary.append(stem(word))

In [55]:
stemmed_dictionary = [i for i in stemmed_dictionary if len(i) > 2]

In [56]:
stemmed_dictionary = list(set(stemmed_dictionary))

In [57]:
stemmed_dictionary.append('хуй')
stemmed_dictionary.append('хуе')

In [58]:
stemmed_dictionary

['сук', 'бля', 'еба', 'пизд', 'хуй', 'хуй', 'хуе']

In [59]:
all_stemmed = dict()
for sentence in df.text:
    for word in set(sentence.split(' ')):
        if word != '':
            if word not in all_stemmed:
                all_stemmed[word] = stem(word)

In [60]:
all_counts = dict()
for sentence in df.text:
    tokens = nltk.word_tokenize(sentence)
    for token in tokens:
        word = stem(token)
        if word in all_counts:
            all_counts[word] += 1
        else:
            all_counts[word] = 1

In [61]:
def get_labled_parts(bad):
    labled_part = dict()
    part = df[df.label == bad]
    for index, row in part.iterrows():
        sentence = row.text
        tokens = nltk.word_tokenize(sentence)
        for token in tokens:
            stemed = stem(token)
            if stemed != '':
                if all_counts[stemed] >= 20:
                    if stemed in labled_part:
                        labled_part[stemed] += 1.0/len(part)
                    else:
                        labled_part[stemed] = 1.0/len(part)
    return labled_part

In [62]:
def sort(x, rev = True):
    return sorted(x.items(), key=operator.itemgetter(1), reverse=rev)

In [63]:
def probability_bad(p_good, p_bad):
    difference = dict()
    for key in p_good:
        if key in p_bad:
            difference[key] =(p_bad[key] - p_good[key])/ np.maximum( p_bad[key], p_good[key]) 
    return difference

In [64]:
def logg_odds(p_good, p_bad):
    ratio = dict()
    for key in p_good:
        if key in p_bad:
            odds_good =  p_good[key]/(1 -  p_good[key])
            odds_bad = p_bad[key]/(1- p_bad[key])
            ratio[key] = np.log(odds_bad/odds_good)
    return ratio

In [65]:
def relable(x):
    global stemmed_dictionary
    tokens = nltk.word_tokenize(x)
    #tokens = [stemmer.stem(token) for token in tokens]
    #if any(bad_word in tokens for bad_word in stemmed_dictionary):
        #return True
    for bad_word in stemmed_dictionary:
        for token in tokens:
            if bad_word in token:
                return True
    return False

In [66]:
def unstem(stem):
    res = []
    for key in all_stemmed:
        if all_stemmed[key] == stem:
            res.append(key)
    return res    

In [67]:
def update_dictionary(word):
    global stemmed_dictionary
    if len(word) <= 2:
        return False
    #for key in stemmed_dictionary:
        #if word in key:
            #return False
    stemmed_dictionary.append(word)
    return True

In [70]:
def iterate(df, report_dictionary, stemmed_dictionary):
    iter = True
    while(iter):
        print('-----------------------------------')
        print('labled as abusive: ', len(df[df.label == True]))
        #print('List of bad_words')
        #print(report_dictionary)
        p_bad = get_labled_parts(True)
        p_good = get_labled_parts(False)
        prob = logg_odds(p_good, p_bad)
        print(sort(prob)[:10])
        print('New bad words')
        new_words = []
        for key in prob:
            if prob[key] > 2.5:
                if update_dictionary(key):
                    new_words += unstem(key)
                    print( unstem(key))
        report_dictionary += new_words
        if len(new_words) == 0:
            iter = False
            break
        #print('\nNew dictionary')
        #print(report_dictionary)
        df['label'] =  df.text.apply(relable)
        print('relabled as abusive: ', len(df[df.label == True]))

        
    

In [71]:
iterate(df, report_dictionary, stemmed_dictionary)

-----------------------------------
labled as abusive:  5753
[('пох', 2.8594630284321307), ('закупа', 2.8150680395931196), ('финлянд', 2.681511285247042), ('погон', 2.586111732042758), ('кароч', 2.5637028872258703), ('уебок', 2.502903943087021), ('драл', 2.502903943087021), ('ебуч', 2.3734582038032257), ('подруг', 2.3690239969560065), ('працюват', 2.3690239969560065)]
New bad words
['финляндии', 'финляндия', 'финляндией', 'финляндию']
['уебок']
['закупаются', 'закупается', 'закупает', 'закупают', 'закупать', 'закупаем']
['драли', 'дрался', 'дрались', 'драл']
['погон', 'погонах', 'погоните', 'погонов', 'погоны', 'погонит', 'погонь', 'погонят']
['карочь', 'кароч', 'кароче']
['пох', 'похую', 'поху', 'похуй', 'похила']
relabled as abusive:  5153
-----------------------------------
labled as abusive:  5153
[('чао', 4.356648880940477), ('нах', 3.590235597320391), ('засад', 3.500348530537158), ('нех', 3.4026268579183667), ('треб', 3.0331698669507445), ('ебуч', 2.49956709536204), ('корабл', 2.

In [39]:
 df['label'] =  df.text.apply(relable)

In [40]:
len(df[df.label == True])

5861

In [33]:
len(stemmed_dictionary)

7

In [155]:
manual['evaluation'] = False

In [72]:
manual['evaluation'] = manual.text.apply(relable)

In [73]:
tp = 0
tn = 0
fp = 0
fn = 0
for index, row in manual.iterrows():
    if row.label == True and row.evaluation == True:
        tp += 1
    if row.label == False and row.evaluation == False:
        tn += 1
    if row.label == False and row.evaluation == True:
        fp += 1
    if row.label == True and row.evaluation == False:
        #print(row.text)
        fn += 1
accuracy = (tp + fp)/(tp + fp + fn + fp)
precision = tp/(tp + fp)
recall = tp/(tp + fn)
f1  = 2 * (precision * recall)/(precision + recall)
print('accuracy: ', accuracy)
print('precision: ', precision)
print('recall: ', recall)
print('f1: ', f1)

accuracy:  0.3701923076923077
precision:  0.7727272727272727
recall:  0.3439306358381503
f1:  0.4760000000000001


In [52]:
fp

8

In [26]:
stemmed_dictionary

['пизд', 'бля', 'еба', 'сук', 'хуй', 'хуе', 'хуё']

In [32]:
update_dictionary('жоп')

True

In [122]:
for sentence in df[df.label == False].text:
    tokens = nltk.word_tokenize(sentence)
    for token in tokens:
        if 'заеба' in tokens:
            print(sentence)

In [109]:
len(df[df.label == False])

40736

In [128]:
c = get_labled_parts(False)

id                UgjG1EIXs7u90HgCoAEC.7-H0Z7-9Z8q7O1Rsamreoe
video_id                                          IC7sx75-A0Q
time                                              2 years ago
author                                                   Эрик
text         евстегнеич ну я же не виноват что у тебя судь...
label                                                   False
Name: 12781, dtype: object
id                                       Ugj4kkvCw5mVwHgCoAEC
video_id                                          0WsJ8-tJeC8
time                                              4 years ago
author                                         Вадим Журавлев
text        националисты майдана выкололи глаз беркутовцу ...
label                                                   False
Name: 39740, dtype: object


In [125]:
c['заеба']

4.909662215239592e-05

In [132]:
df[df.id =='UgjG1EIXs7u90HgCoAEC.7-H0Z7-9Z8q7O1Rsamreoe'].text.values

array([' евстегнеич ну я же не виноват что у тебя судьба такая что ты заёбанным родился это параша каждое утро с колен встаёт после очередной попойки '],
      dtype=object)

In [133]:
 stemmer.stem('заёбанным')

'заеба'